# Assignment #6: Dependency parsing
Author: Pierre Nugues

## Objectives

This assignment is inspired by the CoNLL 2018 shared task of the conference on computational natural language learning on dependency parsing, http://universaldependencies.org/conll18/. It is a follower of <a href="http://ilk.uvt.nl/conll/">CONLL-X</a>, which was the first large-scale evaluation of dependency parsers.
            
In this session, you will implement a dependency parser for Swedish and, optionally, for another language that you will choose.

The objectives of this assignment are to:
* Know what a dependency graph is
* Understand the principles of a transition-based parser
* Extend the parser with a guiding predicate that parses an annotated dependency graph
* Extract features to learn parsing actions from an annotated corpus
* Write a short report on your results

## Organization and location

You can work alone or collaborate with another student.
Each group will have to:
* Write a program that parses a sentence when the dependency graph is known
* Extract features from the parsing actions.
* Train a classifier
* Apply it on a test corpus
* Evaluate the results

## Corpora

As corpora, you will use the Universal Dependencies: https://universaldependencies.org/. The corpora are the same that you used in the 5th assignment, but you will use the test set in addition to the training set. 
1. You will train your parser on training part of the Swedish _Talbanken_ corpus and you will evaluate it on the test set. 
2. Optionally, you will repeat the experiment with another language. 
3. You will comment your results in the report. 

You will load the corpora as in the 5th assignment and the next cells are just copies from this assignment.

### Corpus location

Here are the corpus locations you will use. You may have to adjust `ud_path`.

In [1]:
import os
ud_path = '../../corpus/ud-treebanks-v2.8/'

In [2]:
path_sv_train = ud_path + 'UD_Swedish-Talbanken/sv_talbanken-ud-train.conllu'
path_sv_test = ud_path + 'UD_Swedish-Talbanken/sv_talbanken-ud-test.conllu'

The column names of the CoNLL-U corpora

In [3]:
column_names_u = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC']

#### Functions to read the CoNLL-U files

In [4]:
def read_sentences(file):
    """
    Creates a list of sentences from the corpus
    Each sentence is a string
    :param file:
    :return:
    """
    f = open(file, encoding='utf-8').read().strip()
    sentences = f.split('\n\n')
    return sentences

In [5]:
def split_rows(sentences, column_names):
    """
    Creates a list of sentence where each sentence is a list of lines
    Each line is a dictionary of columns
    :param sentences:
    :param column_names:
    :return:
    """
    new_sentences = []
    root_values = ['0', 'root', 'root', 'root', 'root', 'root', '0', 'root', 'root', 'root']
    start = [dict(zip(column_names, root_values))]
    for sentence in sentences:
        rows = sentence.split('\n')
        sentence = [dict(zip(column_names, row.split('\t'))) for row in rows if row[0] != '#']
        sentence = start + sentence
        new_sentences.append(sentence)
    return new_sentences

#### Reading the corpus

We load the Swedish _Talbanken_ corpus.

In [6]:
sentences = read_sentences(path_sv_train)
formatted_corpus_train = split_rows(sentences, column_names_u)

In [7]:
len(formatted_corpus_train)

4303

The parsed sentence: _Individuell beskattning av arbetsinkomster_

In [8]:
formatted_corpus_train[0]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Individuell',
  'LEMMA': 'individuell',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing',
  'HEAD': '2',
  'DEPREL': 'amod',
  'DEPS': '2:amod',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'beskattning',
  'LEMMA': 'beskattning',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': '0:root',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'av',
  'LEMMA': 'av',
  'UPOS': 'ADP',
  'XPOS': 'PP',
  'FEATS': '_',
  'HEAD': '4',
  'DEPREL': 'case',
  'DEPS': '4:case',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'arbetsinkomster',
  'LEMMA': 'arbetsinkomst',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|PLU|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|G

#### Removing indices that are not integers

To ease the processing of some corpora, we remove the indices which are not integers. We do this because `ID` is not necessarily a number.

In [9]:
def clean_indicies(formatted_corpus):
    formatted_corpus_clean = []
    for sentence in formatted_corpus:
        formatted_corpus_clean.append([word for word in sentence if word['ID'].isdigit()])
    return formatted_corpus_clean          

In [10]:
formatted_corpus_train_clean = clean_indicies(formatted_corpus_train)
formatted_corpus_train_clean[0]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Individuell',
  'LEMMA': 'individuell',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing',
  'HEAD': '2',
  'DEPREL': 'amod',
  'DEPS': '2:amod',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'beskattning',
  'LEMMA': 'beskattning',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': '0:root',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'av',
  'LEMMA': 'av',
  'UPOS': 'ADP',
  'XPOS': 'PP',
  'FEATS': '_',
  'HEAD': '4',
  'DEPREL': 'case',
  'DEPS': '4:case',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'arbetsinkomster',
  'LEMMA': 'arbetsinkomst',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|PLU|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|G

## Transition parser

For each sentence with a projective dependency graph, there is an action sequence that enables the transition parser
to generate this graph. Gold standard parsing corresponds to the sequence of parsing actions, left-arc (<tt>la</tt>), right-arc (<tt>ra</tt>), shift (<tt>sh</tt>), and reduce (<tt>re</tt>) that produces the manually-obtained, gold standard, graph.

### The transitions

Here are implementations of the parsing transitions. Read them and be sure you understand them.

In [11]:
def shift(stack, queue, graph):
    """
    Shift the first word in the queue onto the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    stack = [queue[0]] + stack
    queue = queue[1:]
    return stack, queue, graph


def reduce(stack, queue, graph):
    """
    Remove the first item from the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    return stack[1:], queue, graph


def right_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the top of the stack to the first in the queue
    and shifts
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][queue[0]['ID']] = stack[0]['ID']
    if deprel:
        graph['deprels'][queue[0]['ID']] = deprel
    else:
        graph['deprels'][queue[0]['ID']] = queue[0]['DEPREL']
    # If we create an arc from the 'root', we introduce a statement to pop it to avoid multiple roots
    if stack[0]['ID'] == '0':
        stack = stack[1:]
    return shift(stack, queue, graph)


def left_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the first in the queue to the top of the stack
    and reduces it.
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][stack[0]['ID']] = queue[0]['ID']
    if deprel:
        graph['deprels'][stack[0]['ID']] = deprel
    else:
        graph['deprels'][stack[0]['ID']] = stack[0]['DEPREL']        
    return reduce(stack, queue, graph)

### Constrains on the transitions

We add a few constraints before we carry out the transitions. Given a manually-annotated dependency graph, look at the conditions (`can_...()` functions) on the stack and the current input list -- the queue -- to execute left-arc, right-arc, shift, or reduce. Read about these constrains in the slides.

In [12]:
def can_reduce(stack, graph):
    """
    Checks that the top of the stack has a head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['ID'] in graph['heads']:
        return True
    else:
        return False

    
def can_leftarc(stack, graph):
    """
    Checks that the top of the has no head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['ID'] in graph['heads']:
        return False
    else:
        return True


def can_rightarc(stack):
    """
    Simply checks there is a stack
    :param stack:
    :return:
    """
    if not stack:
        return False
    else:
        return True

### Finding the transitions from a manually-parsed sentence

Using an annotated corpus, we can derive the action sequences producing the manually-parsed sentences (provided that they are projective). We use an oracle for this as explained during the lectures.

In [13]:
def oracle(stack, queue, graph):
    """
    Gold standard parsing
    Produces a sequence of transitions from a manually-annotated corpus:
    sh, re, ra.deprel, la.deprel
    :param stack: The stack
    :param queue: The input list
    :param graph: The set of relations already parsed
    :return: the transition and the grammatical function (deprel) in the
    form of transition.deprel
    """
    # Right arc
    if stack and stack[0]['ID'] == queue[0]['HEAD']:
        # print('ra', queue[0]['DEPREL'], stack[0]['UPOS'], queue[0]['UPOS'])
        deprel = '.' + queue[0]['DEPREL']
        stack, queue, graph = right_arc(stack, queue, graph)
        return stack, queue, graph, 'ra' + deprel
    # Left arc
    if stack and queue[0]['ID'] == stack[0]['HEAD']:
        # print('la', stack[0]['DEPREL'], stack[0]['UPOS'], queue[0]['UPOS'])
        deprel = '.' + stack[0]['DEPREL']
        stack, queue, graph = left_arc(stack, queue, graph)
        return stack, queue, graph, 'la' + deprel
    # Reduce
    if stack and can_reduce(stack, graph):
        for word in stack:
            if (word['ID'] == queue[0]['HEAD'] or
                    word['HEAD'] == queue[0]['ID']):
                # print('re', stack[0]['UPOS'], queue[0]['UPOS'])
                stack, queue, graph = reduce(stack, queue, graph)
                return stack, queue, graph, 're'
    # Shift
    # print('sh', [], queue[0]['UPOS'])
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'

### Dealing with nonprojective graphs

Oracle parsing produces a sequence of transitions if the graph is projective and well-formed. If not, we will have headless words in the stack. Parsing normally terminates when the queue is empty. We also empty the stack to be sure that all the words have a head. We attach headless words to the root word of the sentence.

In [14]:
def exists_root(graph):
    for (x, y)  in graph['heads'].items():
        if y == '0' and x != '0':
            return x
    return False

In [15]:
def empty_stack(stack, graph):
    """
    Pops the items in the stack. If they have no head, they are assigned
    a ROOT head
    :param stack:
    :param graph:
    :return:
    """
    idx_root = exists_root(graph)
    # There is already a root
    if idx_root:
        for word in stack:
            if word['ID'] not in graph['heads']:
                graph['heads'][word['ID']] = idx_root
                graph['deprels'][word['ID']] = 'dep'
    else:
        # There is no root. We assign the root to the first headless word.
        for word in stack:
            if word['ID'] not in graph['heads']:
                if idx_root:
                    graph['heads'][word['ID']] = idx_root
                    graph['deprels'][word['ID']] = 'dep'
                else:
                    graph['heads'][word['ID']] = '0'
                    graph['deprels'][word['ID']] = 'root'
                    idx_root = word['ID']
    stack = []
    return stack, graph

### Checking if two graphs are equal

The `equal_graphs()` utility checks if the graph obtained from a sequence of transitions is equal to the annotated graph. It is normally the case, except with nonprojective graphs.

In [16]:
def equal_graphs(sentence, graph, verbose=False):
    """
    Checks that the graph corresponds to the gold standard annotation of a sentence
    :param sentence:
    :param graph:
    :return:
    """
    equal = True
    for word in sentence:
        if word['ID'] in graph['heads'] and word['HEAD'] == graph['heads'][word['ID']]:
            pass
        else:
            equal = False
            if verbose:
                print(word, flush=True)
    return equal

### Parsing an annotated corpus with an oracle

You will now run the code below. With it, you will produce a sequence of transitions for each sentence. If the graph is projective, applying the sequence to the sentence will recreate the gold-standard annotation.

For this experiment:
1. Understand from the slides used during the lecture how the oracle carries out a gold-standard parsing. 
2. The parser can only deal with projective sentences. In the case of a nonprojective one, the parsed graph and the manually-annotated sentence are not equal. Select one nonprojective sentence (just set `verbose`to `True` in the code below) and examine it. 
3. In your report, you will include one short nonprojective sentence (choose the shortest). You will  **describe** it in the report and you will explain why it is not projective.

In [17]:
def init_config(sentence):
    stack = []
    queue = list(sentence)
    graph = {}
    graph['heads'] = {}
    graph['heads']['0'] = '0'
    graph['deprels'] = {}
    graph['deprels']['0'] = 'ROOT'
    return stack, queue, graph

In [18]:
verbose = False
projectivization = False

transition_corpus = []
graph_corpus = []

for sent_cnt, sentence in enumerate(formatted_corpus_train_clean):
    #print(sentence)
    stack, queue, graph = init_config(sentence)
    transition_sent = []
    while queue:
        stack, queue, graph, trans = oracle(stack, queue, graph)
        transition_sent.append(trans)
    stack, graph = empty_stack(stack, graph)
    transition_corpus.append(transition_sent)
    graph_corpus.append(graph)

    if verbose:
        if not equal_graphs(sentence, graph):
            print('Annotation and gold-standard parsing not equal')
            print('Sentence:', sentence)
            print('Gold-standard graph', graph)
    # Poorman's projectivization to have well-formed graphs.
    # We just just assign the same heads as what gold standard parsing did
    # This guarantee a projective sentence
    if projectivization:
        for word in sentence:
            word['HEAD'] = graph['heads'][word['ID']]
print('\nProcessed ' + str(sent_cnt) + ' sentences')


Processed 4302 sentences


### Checking gold-standard parsing

Apply manually the transition sequence you obtained to the first sentence and check that it parses it correctly. You will draw a stack and a queue with **seven steps** and you will describe this in your report.

In [19]:
formatted_corpus_train_clean[:1]

[[{'ID': '0',
   'FORM': 'root',
   'LEMMA': 'root',
   'UPOS': 'root',
   'XPOS': 'root',
   'FEATS': 'root',
   'HEAD': '0',
   'DEPREL': 'root',
   'DEPS': 'root',
   'MISC': 'root'},
  {'ID': '1',
   'FORM': 'Individuell',
   'LEMMA': 'individuell',
   'UPOS': 'ADJ',
   'XPOS': 'JJ|POS|UTR|SIN|IND|NOM',
   'FEATS': 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing',
   'HEAD': '2',
   'DEPREL': 'amod',
   'DEPS': '2:amod',
   'MISC': '_'},
  {'ID': '2',
   'FORM': 'beskattning',
   'LEMMA': 'beskattning',
   'UPOS': 'NOUN',
   'XPOS': 'NN|UTR|SIN|IND|NOM',
   'FEATS': 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing',
   'HEAD': '0',
   'DEPREL': 'root',
   'DEPS': '0:root',
   'MISC': '_'},
  {'ID': '3',
   'FORM': 'av',
   'LEMMA': 'av',
   'UPOS': 'ADP',
   'XPOS': 'PP',
   'FEATS': '_',
   'HEAD': '4',
   'DEPREL': 'case',
   'DEPS': '4:case',
   'MISC': '_'},
  {'ID': '4',
   'FORM': 'arbetsinkomster',
   'LEMMA': 'arbetsinkomst',
   'UPOS': 'NOUN',
   'XPOS': 'NN|UTR|PLU|

In [20]:
transition_corpus[:1]

[['sh', 'sh', 'la.amod', 'ra.root', 'sh', 'la.case', 'ra.nmod']]

In [21]:
graph_corpus[:1]

[{'heads': {'0': '0', '1': '2', '2': '0', '3': '4', '4': '2'},
  'deprels': {'0': 'ROOT',
   '1': 'amod',
   '2': 'root',
   '3': 'case',
   '4': 'nmod'}}]

## Training a classifier

We can now train a classifier to predict an action from a current parsing context. To be able to predict the next action from a given parsing state, gold-standard parsing must also extract feature vectors at each step of the parsing procedure. The simplest parsing context corresponds to the words' part of speech on the top of the stack and the first of the input list (the queue).
    
Once the data collected, the training procedure will produce a 4-class classifier that you will embed in
Nivre's parser to choose the next action. During parsing, Nivre's parser will call the classifier to choose
the next action in the set {`la`, `ra`, `sh`, `re`} using the current context.

### Feature sets

You will use two feature sets to build your models:
1. The top of the stack and the first word of the input list (word forms and parts of speech);
2. The two first words and POS on the top of the stack and the two first words and POS of the input list;

You will also add constraints to actions. You will encode these constraints as Boolean features.

### Parsing the grammatical functions

Using the actions in the set {`la`, `ra`, `sh`, `re`} produces an unlabelled
graph. It is easy to extend the parser so that it can label the graph with grammatical functions. In this
case, we must complement the actions `la` and `ra` with their function using a notation, `action.function`, where the prefix is the action and the suffix is the function, for example:`la.mod`, `la.case`, `ra.nmod`, etc.

### Extracting features 

The final goal is to parse the Swedish corpus and produce a labelled dependency graph. 

You will train two models with logistic regression using scikit-learn. The two feature sets are:
1. The first set will use the word and the part of speech extracted from the first element in the stack and the first in the queue,
2. the second one will use two elements from the stack and two from the input list (word and part of speech).

These sets will include two additional Boolean parameters, "can do left arc" and "can do reduce", which will model constraints on the parser's actions. In total, the feature sets will then have six, respectively ten parameters.

This means that the purpose of this assignment is to generate two scikit-learn models for the labelled graphs. We use the depth parameter for this: The depth of the stack and the queue, either 1 or 2. Start with 1.

In [22]:
depth = 2

You will need the `queue_stack()` function.

In [23]:
def queue_stack(queue_or_stack, graph, depth, pos=True, lex=True):
    features = []
    features_pos = ['nil'] * depth
    features_lex = ['nil'] * depth
    features_deprel = ['nil'] * depth
    if queue_or_stack:
        for i, word in list(enumerate(queue_or_stack))[:depth]:
            features_pos[i] = queue_or_stack[i]['UPOS']
            features_lex[i] = queue_or_stack[i]['FORM']
    if pos:
        features += features_pos
    if lex:
        features += features_lex
    return features

Optionally, you may want to extend the feature vector with words to the left of the top of the stack with the `right_context()` function. If the top of the stack has index $i$, you will extract the words and their parts of speech at index $i + 1$, $i+2$. This will noticeably improve the performance.

In [24]:
def right_context(stack, sentence, depth, pos=True, lex=True):
    features = []
    features_pos = ['nil'] * depth
    features_lex = ['nil'] * depth
    if stack:
        fw_id = int(stack[0]['ID']) + 1
        for i, word in list(enumerate(sentence))[fw_id: fw_id + depth]:
            features_pos[i - fw_id] = sentence[i]['UPOS']
            features_lex[i - fw_id] = sentence[i]['FORM']
    if pos:
        features += features_pos
    if lex:
        features += features_lex
    return features

The next function returns the features in a dictionary format compatible with scikit-learn. You have a code example of feature encoding in this format in the chunking program.

In [25]:
def extract(depth, stack, queue, graph, sentence):
    """
    :param stack:
    :param queue:
    :param graph:
    :param feature_names:
    :param sentence:
    :return:
    """
    improved = True
    if improved:
        x = (queue_stack(stack, graph, depth) +
             queue_stack(queue, graph, depth) +
             right_context(stack, sentence, depth) +
             [can_reduce(stack, graph), can_leftarc(stack, graph)])
    else:
        x = (queue_stack(stack, graph, depth) +
             queue_stack(queue, graph, depth) +
             [can_reduce(stack, graph), can_leftarc(stack, graph)])     
    feature_names = ['feat' + str(i) for i in range(len(x))]
    features = dict(zip(feature_names, x))
    return features

Now write a loop to parse the annotated corpus using the oracle and collect the features in a matrix ($\mathbf{X}$) and the transitions in a vector ($\mathbf{y}$). 

The first lines of your features for the 4 parameters ($\mathbf{x}$) and labelled actions ($y$) should look like the excerpt below, where the columns correspond to stack0_POS, stack1_POS, stack0_word, stack1_word, queue0_POS, queue1_POS, queue0_word, queue1_word, can-re, can-la, and the transition value (`depth = 2`):
$\mathbf{X} =
\begin{bmatrix}
\text{nil}& \text{nil} &\text{nil} & \text{nil} & \text{ROOT} & \text{ADJ} & \text{ROOT} & \text{Individuell} & \text{False} & \text{False}\\
\text{ROOT} &     \text{nil} &     \text{ROOT} &     \text{nil} &     \text{ADJ} &     \text{NOUN} &     \text{Individuell} &     \text{beskattning} &     \text{True} &     \text{False}\\ 
\text{ADJ} &     \text{ROOT} &     \text{Individuell} &     \text{ROOT} &     \text{NOUN} &     \text{ADP} &     \text{beskattning} &     \text{av} &     \text{False} &     \text{True}\\ 
\text{ROOT} &     \text{nil} &     \text{ROOT} &     \text{nil} &     \text{NOUN} &     \text{ADP} &     \text{beskattning} &     \text{av} &     \text{True} &     \text{False}\\
\text{NOUN} &     \text{ROOT} &     \text{beskattning} &     \text{ROOT} &     \text{ADP} &     \text{NOUN} &     \text{av} &     \text{arbetsinkomster} &     \text{True} &     \text{False}\\
\text{ADP} &     \text{NOUN} &     \text{av} &     \text{beskattning} &     \text{NOUN} &     \text{nil} &     \text{arbetsinkomster} &     \text{nil} &     \text{False} &     \text{True}\\  \text{NOUN} &     \text{ROOT} &     \text{beskattning} &     \text{ROOT} &     \text{NOUN} &     \text{nil} &     \text{arbetsinkomster} &     \text{nil} &     \text{True} &  \text{False}
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
\text{sh}\\
\text{sh}\\
\text{la.amod}\\
\text{ra.root}\\
\text{sh}\\
\text{la.case}\\
\text{ra.nmod}
\end{bmatrix}$

You will store your matrix in a list of Python dictionaries and the classes in a list

In [26]:
X_dict = []
y_symbols = []

In [27]:
# Write your code here


Extracting the features...
0 sentences on 4303
1000 sentences on 4303
2000 sentences on 4303
3000 sentences on 4303
4000 sentences on 4303
Extracting the features: Done


In [28]:
X_dict[:7]

[{'feat0': 'nil',
  'feat1': 'nil',
  'feat2': 'nil',
  'feat3': 'nil',
  'feat4': 'root',
  'feat5': 'ADJ',
  'feat6': 'root',
  'feat7': 'Individuell',
  'feat8': 'nil',
  'feat9': 'nil',
  'feat10': 'nil',
  'feat11': 'nil',
  'feat12': False,
  'feat13': False},
 {'feat0': 'root',
  'feat1': 'nil',
  'feat2': 'root',
  'feat3': 'nil',
  'feat4': 'ADJ',
  'feat5': 'NOUN',
  'feat6': 'Individuell',
  'feat7': 'beskattning',
  'feat8': 'ADJ',
  'feat9': 'NOUN',
  'feat10': 'Individuell',
  'feat11': 'beskattning',
  'feat12': True,
  'feat13': False},
 {'feat0': 'ADJ',
  'feat1': 'root',
  'feat2': 'Individuell',
  'feat3': 'root',
  'feat4': 'NOUN',
  'feat5': 'ADP',
  'feat6': 'beskattning',
  'feat7': 'av',
  'feat8': 'NOUN',
  'feat9': 'ADP',
  'feat10': 'beskattning',
  'feat11': 'av',
  'feat12': False,
  'feat13': True},
 {'feat0': 'root',
  'feat1': 'nil',
  'feat2': 'root',
  'feat3': 'nil',
  'feat4': 'NOUN',
  'feat5': 'ADP',
  'feat6': 'beskattning',
  'feat7': 'av',
  'fe

In [29]:
y_symbols[:7]

['sh', 'sh', 'la.amod', 'ra.root', 'sh', 'la.case', 'ra.nmod']

### Fitting the models

Generate the two scikit-learn models using the code models from the chunking labs.

Vectorize your `X_dict` into an `X` matrix using `DictVectorizer()`

In [30]:
# Write your code here


Fit the model. With sklearn, you can use `y_symbols` directly. Use `verbose=True` and `n_jobs=8` or more.

In [31]:
# Write your code here


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =      4964205     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.43871D+05    |proj g|=  3.87624D+04


 This problem is unconstrained.



At iterate   50    f=  4.10569D+04    |proj g|=  2.74268D+02

At iterate  100    f=  3.18796D+04    |proj g|=  5.35013D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****    100    108      1     0     0   5.350D+01   3.188D+04
  F =   31879.614198967178     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


[Parallel(n_jobs=16)]: Done   1 out of   1 | elapsed:  1.9min finished


## Prediction

Now use this model to predict the sentences in the test corpus

In [32]:
sentences_test = read_sentences(path_sv_test)
formatted_corpus_test = split_rows(sentences_test, column_names_u)
formatted_corpus_test_clean = clean_indicies(formatted_corpus_test)
formatted_corpus_test_clean[0]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Den',
  'LEMMA': 'en',
  'UPOS': 'DET',
  'XPOS': 'DT|UTR|SIN|DEF',
  'FEATS': 'Definite=Def|Gender=Com|Number=Sing|PronType=Art',
  'HEAD': '3',
  'DEPREL': 'det',
  'DEPS': '3:det',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'allmänna',
  'LEMMA': 'allmän',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR/NEU|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing',
  'HEAD': '3',
  'DEPREL': 'amod',
  'DEPS': '3:amod',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'pensionen',
  'LEMMA': 'pension',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Gender=Com|Number=Sing',
  'HEAD': '7',
  'DEPREL': 'nsubj',
  'DEPS': '7:nsubj',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'är',
  'LEMMA': 'vara',
  'UPOS': 'AUX',
  'XPOS': 'VB|PRS|AKT',
  'FEATS': 'Mood=Ind|Tense=

In [33]:
def apply_transition(stack, queue, graph, trans):
    if stack and trans[:2] == 'ra':
        stack, queue, graph = right_arc(stack, queue, graph, trans[3:])
        return stack, queue, graph, 'ra'
    if stack and can_leftarc(stack, graph) and trans[:2] == 'la':
        if trans[3:] == 'root' and exists_root(graph):
            stack, queue, graph = shift(stack, queue, graph)
            return stack, queue, graph, 'sh'
        else:
            stack, queue, graph = left_arc(stack, queue, graph, trans[3:])
            return stack, queue, graph, 'la'
    if stack and can_reduce(stack, graph) and trans == 're':
        stack, queue, graph = reduce(stack, queue, graph)
        return stack, queue, graph, 're'
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'

In [35]:
from tqdm import tqdm

for sent_cnt, sentence in enumerate(tqdm(formatted_corpus_test_clean)):
    X_test_dict = []
    stack, queue, graph = init_config(sentence)
    while queue:
        X_test_dict = extract(depth, stack, queue, graph, sentence)
        X_test = vec.transform(X_test_dict)
        y_test = classifier.predict(X_test)[0]
        stack, queue, graph, trans = apply_transition(stack, queue, graph, y_test)
    stack, graph = empty_stack(stack, graph)
    for word in sentence:
        word['HEAD'] = graph['heads'][word['ID']]
        word['DEPREL'] = graph['deprels'][word['ID']]

 44%|█████████████████▋                      | 540/1219 [04:28<05:57,  1.90it/s]/Users/pierre/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|███████████████████████████████████████| 1219/1219 [10:58<00:00,  1.85it/s]


In [36]:
formatted_corpus_test_clean[0]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'ROOT',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Den',
  'LEMMA': 'en',
  'UPOS': 'DET',
  'XPOS': 'DT|UTR|SIN|DEF',
  'FEATS': 'Definite=Def|Gender=Com|Number=Sing|PronType=Art',
  'HEAD': '3',
  'DEPREL': 'det',
  'DEPS': '3:det',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'allmänna',
  'LEMMA': 'allmän',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR/NEU|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing',
  'HEAD': '3',
  'DEPREL': 'amod',
  'DEPS': '3:amod',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'pensionen',
  'LEMMA': 'pension',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Gender=Com|Number=Sing',
  'HEAD': '7',
  'DEPREL': 'nsubj',
  'DEPS': '7:nsubj',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'är',
  'LEMMA': 'vara',
  'UPOS': 'AUX',
  'XPOS': 'VB|PRS|AKT',
  'FEATS': 'Mood=Ind|Tense=

In [37]:
formatted_corpus_test[0]

[{'ID': '0',
  'FORM': 'root',
  'LEMMA': 'root',
  'UPOS': 'root',
  'XPOS': 'root',
  'FEATS': 'root',
  'HEAD': '0',
  'DEPREL': 'ROOT',
  'DEPS': 'root',
  'MISC': 'root'},
 {'ID': '1',
  'FORM': 'Den',
  'LEMMA': 'en',
  'UPOS': 'DET',
  'XPOS': 'DT|UTR|SIN|DEF',
  'FEATS': 'Definite=Def|Gender=Com|Number=Sing|PronType=Art',
  'HEAD': '3',
  'DEPREL': 'det',
  'DEPS': '3:det',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'allmänna',
  'LEMMA': 'allmän',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR/NEU|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing',
  'HEAD': '3',
  'DEPREL': 'amod',
  'DEPS': '3:amod',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'pensionen',
  'LEMMA': 'pension',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Gender=Com|Number=Sing',
  'HEAD': '7',
  'DEPREL': 'nsubj',
  'DEPS': '7:nsubj',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'är',
  'LEMMA': 'vara',
  'UPOS': 'AUX',
  'XPOS': 'VB|PRS|AKT',
  'FEATS': 'Mood=Ind|Tense=

In [38]:
len(formatted_corpus_test_clean)

1219

In [39]:
def save(file, formatted_corpus, column_names):
    f_out = open(file, 'w', encoding='utf-8')
    for sentence in formatted_corpus:
        for row in sentence[1:]:
            # print(row, flush=True)
            for col in column_names[:-1]:
                if col in row:
                    f_out.write(row[col] + '\t')
                else:
                    f_out.write('_\t')
            col = column_names[-1]
            if col in row:
                f_out.write(row[col] + '\n')
            else:
                f_out.write('_\n')
        f_out.write('\n')
    #f_out.write('\n')
    f_out.close()

In [40]:
out_file_name = 'out_sys'
save(out_file_name, formatted_corpus_test_clean, column_names_u)

### Evaluation

1. Once you have parsed the test set, you will measure the accuracy of your parser using the CoNLL evaluation script available here: http://universaldependencies.org/conll18/evaluation.html. Download it.
2. You will run the evaluation in the cell below. Be sure to have the Python program in your folder.
2. You will report your best score. You need to reach a _labelled attachment score_ (LAS) of 0.67 to pass this lab.

In [41]:
import conll18_ud_eval
system_ud_file = open(out_file_name, encoding='utf-8')
system_ud = conll18_ud_eval.load_conllu(system_ud_file)

gold_ud_file = open(path_sv_test, encoding='utf-8')
gold_ud = conll18_ud_eval.load_conllu(gold_ud_file)

las = conll18_ud_eval.evaluate(gold_ud, system_ud)['LAS'].f1
las

0.7553614369141679

Should you want to run the script in other experiments, just execute: `python conll18_ud_eval.py gold_file system_file`

### Reading

Read the article: _Globally Normalized Transition-Based Neural Networks_ by Andor and al. (2016) [<a href="https://www.aclweb.org/anthology/P16-1231">pdf</a>] and write in a few sentences how it relates to your work in this assignment.</p>

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [42]:
STIL_ID = ["student1", "student2"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "6-dependency_parsing_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of your best labeled attachment score.

In [43]:
import json
ANSWER = json.dumps({'las': las
                    })
ANSWER

'{"las": 0.7553614369141679}'

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [44]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [45]:
import bz2
ASSIGNMENT = 6
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [46]:
import requests
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
               verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': '9f56a1ad60b611212a876d72a017ccc2d811aebef71828376d64dccc1c1c81bdc56b659cd17cb130e12c1baf5238334fb87d7427fa89e2dfcce3e367f5af6bea',
 'submission_id': '7b5bfd6a-9dbe-4d6a-8adb-53cf32ab50ac'}